# <center>CryptoCurrencies CA#2<center>
<h1><center>Transacting on Bitcoin TestNet / Firing up an Ethereum Node<center></h1>

#### <center>"Morteza - Rashidkhan"<center>
#### <center>"810899055"<center>

In [2]:
!pip install base58 ecdsa ripemd-hash python-bitcoinlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.3/149.3 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 11.7 MB/s eta 0:00:00
  Created wheel for ripemd-hash: filename=ripemd_hash-1.0.1-cp310-cp310-linux_x86_64.whl size=19072 sha256=863fdfdfc6dcc0b048c504467be69c55264e06fd2abf6815086c7ee7865149e5
  Stored in directory: /root/.cache/pip/wheels/bd/e3/8d/d140830b288c0694608cd501a7cde6c1d6abf754d37d57cd6d
Successfully built ripemd-hash


In [3]:
import os
import base58
import ecdsa
from hashlib import sha256
import binascii
from ripemd.ripemd160 import ripemd160

## 1. Address Generation:

In [4]:
def getWif(privkey):
    wif = b"\xef" + privkey # 0xef for testnet and 0x80 for mainnet
    checksum = sha256(sha256(wif).digest()).digest()[:4]
    wif += checksum
    return base58.b58encode(wif)

 1.1: Base58 and WIF address

In [157]:
def btc_add_generate():
  ecdsaPrivateKey = ecdsa.SigningKey.generate(curve=ecdsa.SECP256k1)
  # print("ECDSA Private Key: ", ecdsaPrivateKey.to_string().hex())

  ecdsaPublicKey = '04' +  ecdsaPrivateKey.get_verifying_key().to_string().hex()
  # print("ECDSA Public Key: ", ecdsaPublicKey)

  hash256FromECDSAPublicKey = sha256(binascii.unhexlify(ecdsaPublicKey)).hexdigest()
  # print("SHA256(ECDSA Public Key): ", hash256FromECDSAPublicKey)

  ridemp160FromHash256 = ripemd160(binascii.unhexlify(hash256FromECDSAPublicKey))
  # print("RIDEMP160(SHA256(ECDSA Public Key)): ", ridemp160FromHash256.hex())

  prependNetworkByte = '6f' + ridemp160FromHash256.hex() # 0x6f for testnet and 0x00 for mainnet
  # print("Prepend Network Byte to RIDEMP160(SHA256(ECDSA Public Key)): ", prependNetworkByte)

  hash = prependNetworkByte
  for x in range(1,3):
      hash = sha256(binascii.unhexlify(hash)).hexdigest()
      # print("\t|___>SHA256 #", x, " : ", hash)

  cheksum = hash[:8]
  # print("Checksum(first 4 bytes): ", cheksum)

  appendChecksum = prependNetworkByte + cheksum
  # print("Append Checksum to RIDEMP160(SHA256(ECDSA Public Key)): ", appendChecksum)

  bitcoinAddress = base58.b58encode(binascii.unhexlify(appendChecksum))
  # print("Bitcoin Address: ", bitcoinAddress.decode('utf8'))

  return bitcoinAddress, getWif(ecdsaPrivateKey.to_string())

address, wif = btc_add_generate()
print(f"bitcoin address: {address.decode()}")
print(f"Private Key (WIF): {wif.decode()}")

bitcoin address: n2px8MPScKUkhiWZxckkTHZQVda14g4svz
Private Key (WIF): 92ABWZK4kLxN1EqbkPLQcUntwm5ZfLE4SxR6o8WQSfHSXLjwJZc


1.2: Vanity Address Generator


In [ ]:
def find_vanity_address(prefix):
  while True:
    ecdsaPrivateKey = ecdsa.SigningKey.generate(curve=ecdsa.SECP256k1)
    ecdsaPublicKey = b'\x04' +  ecdsaPrivateKey.get_verifying_key().to_string()
    hash256FromECDSAPublicKey = sha256((ecdsaPublicKey)).digest()
    ridemp160FromHash256 = ripemd160(hash256FromECDSAPublicKey)
    prependNetworkByte = b'\x6f' + ridemp160FromHash256 # 0x6f for testnet and 0x00 for mainnet
    hash = prependNetworkByte
    for x in range(1,3):
        hash = sha256(hash).digest()
    cheksum = hash[:4]
    appendChecksum = prependNetworkByte + cheksum
    bitcoinAddress = base58.b58encode(appendChecksum)
    if bitcoinAddress.decode()[1:].startswith(prefix):
      return bitcoinAddress, getWif(ecdsaPrivateKey.to_string())

desired_prefix = "pik"
vanity_address, vanity_wif = find_vanity_address(desired_prefix)
print(f"Vanity Address: {vanity_address.decode()}")
print(f"Private Key (WIF): {vanity_wif.decode()}")

Vanity Address: mpikALrCM4VDR6gmd3K7638pGQALyd32xK
Private Key (WIF): 92ccUJ7FUaw7vPUKfXTmBF5k1DBJ77GWify7N9sEQuTxunsKcHk


## 2. Transacting on Bitcoin TestNet:

Here are some utility functions that you will need to use in order to make transactions. Fill the #TODO parts in the functions in order to be used in the next steps:

#### Utility Functions:

In [4]:
import bitcoin.wallet
from bitcoin.core import COIN, b2lx, serialize, x, lx, b2x
from bitcoin.core.scripteval import OP_DUP, OP_HASH160, OP_EQUALVERIFY, OP_CHECKSIG
# from utils import *

bitcoin.SelectParams("testnet")  # Select the network (testnet or mainnet)
# Private key in WIF format XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
my_private_key = bitcoin.wallet.CBitcoinSecret("92b2YxT5wqFtT8oZJqd91P6n4rWonFg7XBPPRgegt6szp4dgMY9")
my_public_key = my_private_key.pub
my_address = bitcoin.wallet.P2PKHBitcoinAddress.from_pubkey(my_public_key)
destination_address = bitcoin.wallet.CBitcoinAddress("mpikWc4XVfcaYqVUZ4bk9zkzHWv8M2PxQG")  # Destination address (recipient of the money)


def P2PKH_scriptPubKey(address):
    decoded = base58.b58decode(address)
    publicKeyHash = decoded[1:-4]
    return [OP_DUP, OP_HASH160, publicKeyHash, OP_EQUALVERIFY, OP_CHECKSIG]


def P2PKH_scriptSig(txin, txout, txin_scriptPubKey):
    signature = create_OP_CHECKSIG_signature(txin, txout, txin_scriptPubKey, my_private_key)

    return [signature, my_public_key]  # Fill this section


def send_from_P2PKH_transaction(amount_to_send, txid_to_spend, utxo_index, txout_scriptPubKey):
    txout = create_txout(amount_to_send, txout_scriptPubKey)

    txin_scriptPubKey = P2PKH_scriptPubKey(my_address)
    txin = create_txin(txid_to_spend, utxo_index)
    txin_scriptSig = P2PKH_scriptSig(txin, txout, txin_scriptPubKey)

    new_tx = create_signed_transaction(txin, txout, txin_scriptPubKey, txin_scriptSig)

    return broadcast_transaction(new_tx)


def run():
    ######################################################################
    amount_to_send = 0.1
    txid_to_spend = ('ad310edb4ff2cb6a4cd2c7de13c5ab321ac0f90022170a109c84836f6ce54abc')  # TxHash of UTXO
    utxo_index = 0  # UTXO index among transaction outputs
    ######################################################################

    print(my_address)  # Prints your address in base58
    print(my_public_key.hex())  # Print your public key in hex
    print(my_private_key.hex())  # Print your private key in hex
    txout_scriptPubKey = P2PKH_scriptPubKey(my_address)
    response = send_from_P2PKH_transaction(amount_to_send, txid_to_spend, utxo_index, txout_scriptPubKey)
    print(response.status_code, response.reason)
    print(response.text)  # Report the hash of transaction which is printed in this section result


In [5]:
import requests

from bitcoin.core import b2x, lx, COIN, COutPoint, CMutableTxOut, CMutableTxIn, CMutableTransaction, Hash160
from bitcoin.core.script import *
from bitcoin.core.scripteval import VerifyScript, SCRIPT_VERIFY_P2SH


def send_from_custom_transaction(amount_to_send, txid_to_spend, utxo_index, txin_scriptPubKey, txin_scriptSig, txout_scriptPubKey):
    txout = create_txout(amount_to_send, txout_scriptPubKey)
    txin = create_txin(txid_to_spend, utxo_index)
    new_tx = create_signed_transaction(txin, txout, txin_scriptPubKey, txin_scriptSig)
    return broadcast_transaction(new_tx)


def create_txin(txid, utxo_index):
    return CMutableTxIn(COutPoint(lx(txid), utxo_index))


def create_txout(amount, scriptPubKey):
    return CMutableTxOut(amount*COIN, CScript(scriptPubKey))


def create_OP_CHECKSIG_signature(txin, txout, txin_scriptPubKey, seckey):
    tx = CMutableTransaction([txin], [txout])
    sighash = SignatureHash(CScript(txin_scriptPubKey), tx, 0, SIGHASH_ALL)
    sig = seckey.sign(sighash) + bytes([SIGHASH_ALL])
    return sig


def create_signed_transaction(txin, txout, txin_scriptPubKey, txin_scriptSig):
    tx = CMutableTransaction([txin], [txout])
    txin.scriptSig = CScript(txin_scriptSig)
    VerifyScript(txin.scriptSig, CScript(txin_scriptPubKey), tx, 0, (SCRIPT_VERIFY_P2SH,))
    return tx


def broadcast_transaction(tx):
    raw_transaction = b2x(tx.serialize())
    headers = {'content-type': 'application/x-www-form-urlencoded'}
    return requests.post(
        'https://mempool.space/testnet4/api/tx',
        headers=headers,
        data='{"tx": "%s"}' % raw_transaction,
    )


Now use the provided functions to make these transactions:

### **2.1: One input and two outputs transaction**

In [ ]:
def create_first_transaction():
    txid_to_spend = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'  # Replace with actual txid
    utxo_index = 1
    amount_to_send = 0.00560572  # Amount in BTC
    fee = 0.0001  # Fee in BTC

    private_key = bitcoin.wallet.CBitcoinSecret("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx") # Replace with actual private key
    public_key = private_key.pub
    address = bitcoin.wallet.P2PKHBitcoinAddress.from_pubkey(public_key)

    hashed_true = Hash160(b'\x51') # OP_TRUE

    # Create outputs
    unspendable_scriptPubKey = [OP_RETURN, b'un_spendable']
    spendable_scriptPubKey = [OP_HASH160, hashed_true, OP_EQUAL]

    txin = create_txin(txid_to_spend, utxo_index)
    txout1 = create_txout(0.0, unspendable_scriptPubKey)  # 0 BTC output to OP_RETURN
    txout2 = create_txout(amount_to_send - fee, spendable_scriptPubKey)

    txin_scriptPubKey = P2PKH_scriptPubKey(str(address))

    tx = CMutableTransaction([txin], [txout1, txout2])
    sighash = SignatureHash(CScript(txin_scriptPubKey), tx, 0, SIGHASH_ALL)
    sig = private_key.sign(sighash) + bytes([SIGHASH_ALL])

    txin_scriptSig = [sig, public_key]
    txin.scriptSig = CScript(txin_scriptSig)
    VerifyScript(txin.scriptSig, CScript(txin_scriptPubKey), tx, 0, (SCRIPT_VERIFY_P2SH,))

    # response = broadcast_transaction(tx)
    # print(response.status_code, response.reason)
    # print(response.text)
    return b2x(tx.serialize())


def create_second_transaction(txid_to_spend):
    # Replace with the txid of the first transaction and the correct output index
    utxo_index = 1  # The index of the spendable output in the first transaction
    amount_to_send = 0.00550572  # Amount in BTC (after subtracting fee)
    fee = 0.00001  # Fee in BTC

    private_key = bitcoin.wallet.CBitcoinSecret("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")  # Replace with actual private key
    public_key = private_key.pub
    address = bitcoin.wallet.P2PKHBitcoinAddress.from_pubkey(public_key)

    txin = create_txin(txid_to_spend, utxo_index)
    txout_scriptPubKey = P2PKH_scriptPubKey(str(address))

    txout = create_txout(amount_to_send - fee, txout_scriptPubKey)
    tx = CMutableTransaction([txin], [txout])

    # Previous scriptPubKey
    txin_scriptPubKey = [0x51]
    # No signature needed as it is spendable by anyone
    txin_scriptSig = [0x51]

    txin.scriptSig = CScript(txin_scriptSig)
    VerifyScript(txin.scriptSig, CScript(txin_scriptPubKey), tx, 0, (SCRIPT_VERIFY_P2SH,))

    # response = broadcast_transaction(tx)
    # print(response.status_code, response.reason)
    # print(response.text)
    return b2x(tx.serialize())


**first transaction:**

In [ ]:
tx1_hex1 = create_first_transaction()
# !echo $tx1_hex1
!curl -X POST -sSLd "$tx1_hex1" "https://mempool.space/testnet4/api/tx"

**second transaction:**

In [ ]:
tx1_hex2 = create_second_transaction("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx") ## txid from first transaction
# !echo $tx1_hex2
!curl -X POST -sSLd "$tx1_hex2" "https://mempool.space/testnet4/api/tx"

### 2.2: Pay to MultiSig transaction (P2MS)


In [6]:
# Create the conditional scriptPubKey
def create_P2MS(pubkey1, pubkey2, pubkey3):
    return [OP_2, pubkey1, pubkey2, pubkey3, OP_3, OP_CHECKMULTISIG]

def create_P2MS_transaction(txid_to_spend, utxo_index, amount_to_send, fee, dst_pubkey1, dst_pubkey2, dst_pubkey3, my_secKey, txin_scriptPubKey):
    txin = CMutableTxIn(COutPoint(lx(txid_to_spend), utxo_index))
    txout_scriptPubKey = create_P2MS(dst_pubkey1, dst_pubkey2, dst_pubkey3)
    txout = CMutableTxOut((amount_to_send - fee) * COIN, CScript(txout_scriptPubKey))
    tx = CMutableTransaction([txin], [txout])

    # Sign the transaction
    sighash = SignatureHash(CScript(txin_scriptPubKey), tx, 0, SIGHASH_ALL)
    sig = my_secKey.sign(sighash) + bytes([SIGHASH_ALL])

    txin.scriptSig = CScript([sig, my_secKey.pub])

    VerifyScript(txin.scriptSig, CScript(txin_scriptPubKey), tx, 0, (SCRIPT_VERIFY_P2SH,))

    raw_transaction = b2x(tx.serialize())
    return raw_transaction

def spend_P2MS_transaction(txid_to_spend, utxo_index, amount_to_send, fee, seckey1, seckey2, my_address, txin_scriptPubKey):
  txin = CMutableTxIn(COutPoint(lx(txid_to_spend), utxo_index))
  txout_scriptPubKey = P2PKH_scriptPubKey(str(my_address))
  txout = CMutableTxOut((amount_to_send - fee) * COIN, CScript(txout_scriptPubKey))
  tx = CMutableTransaction([txin], [txout])

  # Sign the transaction
  sighash = SignatureHash(CScript(txin_scriptPubKey), tx, 0, SIGHASH_ALL)

  sig1 = seckey1.sign(sighash) + bytes([SIGHASH_ALL])
  sig2 = seckey2.sign(sighash) + bytes([SIGHASH_ALL])

  txin.scriptSig = CScript([0, sig1, sig2])

  VerifyScript(txin.scriptSig, CScript(txin_scriptPubKey), tx, 0, (SCRIPT_VERIFY_P2SH,))

  raw_transaction = b2x(tx.serialize())
  return raw_transaction

#### **Create P2MS Transaction**

In [167]:
my_private_key = bitcoin.wallet.CBitcoinSecret("92yyk3sjqzTTXoTAk9XcHPvxB3pFdiLGF5bDzZTr7GJUSwGCEQC") # Replace with actual wif
my_public_key = my_private_key.pub
my_address = bitcoin.wallet.P2PKHBitcoinAddress.from_pubkey(my_public_key)

dst_seckey1 = bitcoin.wallet.CBitcoinSecret("93N2Hw71bHAcvkn4JM3VLqW8e3MYJRyoDVe5aXdbvmh7XCX5Fyz") # Replace with actual destination wif
dst_pubkey1 = dst_seckey1.pub
dst_address1 = bitcoin.wallet.P2PKHBitcoinAddress.from_pubkey(dst_pubkey1)

dst_seckey2 = bitcoin.wallet.CBitcoinSecret("93Q5PZFgZfM9EcuDHf5Q8ASrv4K53QNWMkiVPFPtJmUQA6pWK5s") # Replace with actual destination wif
dst_pubkey2 = dst_seckey2.pub
dst_address2 = bitcoin.wallet.P2PKHBitcoinAddress.from_pubkey(dst_pubkey2)

dst_seckey3 = bitcoin.wallet.CBitcoinSecret("92ABWZK4kLxN1EqbkPLQcUntwm5ZfLE4SxR6o8WQSfHSXLjwJZc") # Replace with actual destination wif
dst_pubkey3 = dst_seckey3.pub
dst_address3 = bitcoin.wallet.P2PKHBitcoinAddress.from_pubkey(dst_pubkey3)

txid = 'ff5ad8d5b0cd7b0489efe84ce018b4f2529559f6b3d0fb1e378b718d835d0821'  # Replace with actual txid from faucet
utxo_index = 0
amount_to_send = 0.00487645  ## replace with your actual balance
fee = 0.0001

txin_scriptPubKey = P2PKH_scriptPubKey(str(my_address))
tx2_hex1 = create_P2MS_transaction(txid, utxo_index, amount_to_send, fee, dst_pubkey1, dst_pubkey2, dst_pubkey3, my_private_key, txin_scriptPubKey)
# !echo $tx2_hex1
!curl -X POST -sSLd "$tx2_hex1" "https://mempool.space/testnet4/api/tx"

010000000121085d838d718b371efbd0b3f6599552f2b418e04ce8ef89047bcdb0d5d85aff000000008b4830450221009fea0363ee3821ecdd957914194e18d41ff04092412353772efd2562b668dc4a02204bf27fbe053c2f5f96951b1e105de183df62960ce5302a1b608af17e1b96619401410461aabb2ab8fbf76f7d5b5701bfdaba4c04be33722a5cc3d3c331a95fc4cd1c45b49deedba6907d46e53d0f70116a1ea60c6f700f99a3c226db280aa6ec1b77d2ffffffff01cd49070000000000c9524104ad34a2c1bbd3aec7ebae0c3cfab37c0715ec3a189597ae31b1ed1f44abe93047e2ec7a945c2a1219484bdb458068bb8a7ce13c190325357a29424a089b8bd756410478607280574ccab25285b26d225c02988b68cf2adead05f2d21a12b3006026d6e71aa2491733c8731d4ac44be2ae5eb4552180c9d0cb29f37fb0167adb51b37e4104bf81ac047f76bd187351a9dc5ea2fead1b0de39fc367e9b6ebdcc1d877dfb2da8ec28ad50dde6732dc94bdd4f26382bac4f69cda10987b43151cb613f7e06f7653ae00000000


#### **Spend P2MS Transaction**

In [7]:
my_private_key = bitcoin.wallet.CBitcoinSecret("92yyk3sjqzTTXoTAk9XcHPvxB3pFdiLGF5bDzZTr7GJUSwGCEQC") # Replace with actual wif
my_public_key = my_private_key.pub
my_address = bitcoin.wallet.P2PKHBitcoinAddress.from_pubkey(my_public_key)

dst_seckey1 = bitcoin.wallet.CBitcoinSecret("93N2Hw71bHAcvkn4JM3VLqW8e3MYJRyoDVe5aXdbvmh7XCX5Fyz") # Replace with actual destination wif
dst_pubkey1 = dst_seckey1.pub
dst_address1 = bitcoin.wallet.P2PKHBitcoinAddress.from_pubkey(dst_pubkey1)

dst_seckey2 = bitcoin.wallet.CBitcoinSecret("93Q5PZFgZfM9EcuDHf5Q8ASrv4K53QNWMkiVPFPtJmUQA6pWK5s") # Replace with actual destination wif
dst_pubkey2 = dst_seckey2.pub
dst_address2 = bitcoin.wallet.P2PKHBitcoinAddress.from_pubkey(dst_pubkey2)

dst_seckey3 = bitcoin.wallet.CBitcoinSecret("92ABWZK4kLxN1EqbkPLQcUntwm5ZfLE4SxR6o8WQSfHSXLjwJZc") # Replace with actual destination wif
dst_pubkey3 = dst_seckey3.pub
dst_address3 = bitcoin.wallet.P2PKHBitcoinAddress.from_pubkey(dst_pubkey3)

txid = '21f4713326dbe56bdd553613fdf6f112086425f55c83fd54e8a2f36045e1d965'  # Replace with actual txid from faucet
utxo_index = 0
amount_to_send = 0.00477645  ## replace with your actual balance
fee = 0.0001

txin_scriptPubKey = create_P2MS(dst_pubkey1, dst_pubkey2, dst_pubkey3)
tx2_hex1 = spend_P2MS_transaction(txid, utxo_index, amount_to_send, fee, dst_seckey1, dst_seckey2, my_address, txin_scriptPubKey)
# !echo $tx2_hex1
!curl -X POST -sSLd "$tx2_hex1" "https://mempool.space/testnet4/api/tx"

f2b2a965cac99c85f71f8705454793183e93a47b558c485dec92c1101bdacf55

### 2.3: Custom Transaction (BitCoin Scripting)


In [144]:
# Create the conditional scriptPubKey
def create_redeem_script(public_key):
    hashed_password = sha256(b'StudentId').digest()  ## replace with actual student id
    current_year = 0x7E8  ## 2024 = 0x7E8
    legal_age = 0x12 ## 18 = 0x12
    return [OP_IF,
            current_year, OP_SWAP, OP_SUB, legal_age, OP_GREATERTHAN, OP_VERIFY,
        OP_ELSE,
            OP_SHA256, hashed_password, OP_EQUALVERIFY,
        OP_ENDIF,
        public_key, OP_CHECKSIG
    ]

def create_lock_script(redeem_script):
    return [OP_HASH160, Hash160(CScript(redeem_script)), OP_EQUAL]

def create_initial_transaction(txid_to_spend, utxo_index, amount_to_send, fee, dst_pubkey, my_secKey, txin_scriptPubKey):
    txin = CMutableTxIn(COutPoint(lx(txid_to_spend), utxo_index))
    redeem_script = create_redeem_script(dst_pubkey)
    lock_script = create_lock_script(redeem_script)
    txout = CMutableTxOut((amount_to_send - fee) * COIN, CScript(lock_script))
    tx = CMutableTransaction([txin], [txout])

    # Sign the transaction
    sighash = SignatureHash(CScript(txin_scriptPubKey), tx, 0, SIGHASH_ALL)
    sig = my_secKey.sign(sighash) + bytes([SIGHASH_ALL])

    txin.scriptSig = CScript([sig, my_secKey.pub])

    VerifyScript(txin.scriptSig, CScript(txin_scriptPubKey), tx, 0, (SCRIPT_VERIFY_P2SH,))

    raw_transaction = b2x(tx.serialize())
    return raw_transaction, lock_script

def spend_by_verifying_age(txid_to_spend, utxo_index, amount_to_send, fee, birth_year, dst_address, my_secKey, lock_script):
    txin = CMutableTxIn(COutPoint(lx(txid_to_spend), utxo_index))
    txout = CMutableTxOut((amount_to_send - fee) * COIN, CScript(P2PKH_scriptPubKey(str(dst_address))))

    redeem_script = create_redeem_script(my_secKey.pub)

    tx = CMutableTransaction([txin], [txout])
    sighash = SignatureHash(CScript(redeem_script), tx, 0, SIGHASH_ALL)
    sig = my_secKey.sign(sighash) + bytes([SIGHASH_ALL])

    # Create scriptSig
    unlock_script = [sig, birth_year, OP_TRUE, CScript(redeem_script)]
    txin.scriptSig = CScript(unlock_script)


    VerifyScript(txin.scriptSig, CScript(lock_script), tx, 0, (SCRIPT_VERIFY_P2SH,))
    raw_transaction = b2x(tx.serialize())
    return raw_transaction


def spend_by_providing_password(txid_to_spend, utxo_index, amount_to_send, fee, password, dst_address, my_secKey, lock_script):
    txin = CMutableTxIn(COutPoint(lx(txid_to_spend), utxo_index))
    txout = CMutableTxOut((amount_to_send - fee) * COIN, CScript(P2PKH_scriptPubKey(str(dst_address))))

    redeem_script = create_redeem_script(my_secKey.pub)

    tx = CMutableTransaction([txin], [txout])
    sighash = SignatureHash(CScript(redeem_script), tx, 0, SIGHASH_ALL)
    sig = my_secKey.sign(sighash) + bytes([SIGHASH_ALL])

    # Create scriptSig
    txin_scriptSig = [sig, password, OP_FALSE, CScript(redeem_script)]

    txin.scriptSig = CScript(txin_scriptSig)
    VerifyScript(txin.scriptSig, CScript(lock_script), tx, 0, (SCRIPT_VERIFY_P2SH,))


    raw_transaction = b2x(tx.serialize())
    return raw_transaction

#### **BY BIRTH YEAR**

**create first transaction**

In [146]:
my_private_key = bitcoin.wallet.CBitcoinSecret("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx") # Replace with actual wif
my_public_key = my_private_key.pub
my_address = bitcoin.wallet.P2PKHBitcoinAddress.from_pubkey(my_public_key)

dst_seckey = bitcoin.wallet.CBitcoinSecret("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx") # Replace with actual destination wif
dst_pubkey = dst_seckey.pub
dst_address = bitcoin.wallet.P2PKHBitcoinAddress.from_pubkey(dst_pubkey)

txid = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'  # Replace with actual txid from faucet
utxo_index = 0
amount_to_send = 0.001  ## replace with your actual balance
fee = 0.0001

txin_scriptPubKey = P2PKH_scriptPubKey(str(my_address))
tx1_hex, lock_script1 = create_initial_transaction(txid, utxo_index, amount_to_send, fee, dst_pubkey, my_private_key, txin_scriptPubKey)
# !echo $tx1_hex
!curl -X POST -sSLd "$tx1_hex" "https://mempool.space/testnet4/api/tx"

f0c813582d132e1e235aec1f1f905fbc0e1cf504a71ae887caeac8a274982443

**spend above transaction by birth year**

In [152]:
my_private_key = bitcoin.wallet.CBitcoinSecret("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx") # Replace with actual wif
my_public_key = my_private_key.pub
my_address = bitcoin.wallet.P2PKHBitcoinAddress.from_pubkey(my_public_key)

dst_seckey = bitcoin.wallet.CBitcoinSecret("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx") # Replace with actual destination wif
dst_pubkey = dst_seckey.pub
dst_address = bitcoin.wallet.P2PKHBitcoinAddress.from_pubkey(dst_pubkey)

txid = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'  # Replace with actual txid from faucet
utxo_index = 0
amount_to_send = 0.001  ## replace with your actual balance
fee = 0.0001

txin_scriptPubKey = lock_script1
birth_year = 0x7D1 ## 2001 = 0x7D1

tx_spend1_hex = spend_by_verifying_age(txid, utxo_index, amount_to_send, fee, birth_year, dst_address, my_private_key, txin_scriptPubKey)

# !echo $tx_spend1_hex
!curl -X POST -sSLd "$tx_spend1_hex" "https://mempool.space/testnet4/api/tx"

2a691884927c92649b0c8759f929b931ba21d75bb21bc21f4a3b5868be0bc4d7

#### **BY PASSWORD**

**create first transaction**

In [12]:
my_private_key = bitcoin.wallet.CBitcoinSecret("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx") # Replace with actual wif
my_public_key = my_private_key.pub
my_address = bitcoin.wallet.P2PKHBitcoinAddress.from_pubkey(my_public_key)

dst_seckey = bitcoin.wallet.CBitcoinSecret("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx") # Replace with actual destination wif
dst_pubkey = dst_seckey.pub
dst_address = bitcoin.wallet.P2PKHBitcoinAddress.from_pubkey(dst_pubkey)

txid = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'  # Replace with actual txid from faucet
utxo_index = 0
amount_to_send = 0.001  ## replace with your actual balance
fee = 0.0001

txin_scriptPubKey = P2PKH_scriptPubKey(str(my_address))
tx1_hex, lock_script1 = create_initial_transaction(txid, utxo_index, amount_to_send, fee, dst_pubkey, my_private_key, txin_scriptPubKey)
# !echo $tx1_hex
!curl -X POST -sSLd "$tx1_hex" "https://mempool.space/testnet4/api/tx"

52ceb80c5c7a5a0eb6bd2fe84de9aaf119b6339e74648e300c3175ae0174c7d5

**spend above transaction by password**

In [128]:
my_private_key = bitcoin.wallet.CBitcoinSecret("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx") # Replace with actual wif
my_public_key = my_private_key.pub
my_address = bitcoin.wallet.P2PKHBitcoinAddress.from_pubkey(my_public_key)

dst_seckey = bitcoin.wallet.CBitcoinSecret("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx") # Replace with actual destination wif
dst_pubkey = dst_seckey.pub
dst_address = bitcoin.wallet.P2PKHBitcoinAddress.from_pubkey(dst_pubkey)

txid = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'  # Replace with actual txid from faucet
utxo_index = 0
amount_to_send = 0.001  ## replace with your actual balance
fee = 0.0001

password = b'xxxxxxxxxx' ## replace with your password

txin_scriptPubKey = lock_script1

tx_spend1_hex = spend_by_providing_password(txid, utxo_index, amount_to_send, fee, password, dst_address, my_private_key, txin_scriptPubKey)

# !echo $tx_spend1_hex
!curl -X POST -sSLd "$tx_spend1_hex" "https://mempool.space/testnet4/api/tx"

d1a68c8f20cc0ce8297e4f4b5ec297af1c6f98630e8105fd9d63b39c004c4ff0